In [1]:
# Importing libraries

from __future__ import print_function
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import classification_report
from sklearn import metrics
from sklearn import tree
from sklearn.preprocessing import LabelEncoder
import warnings
warnings.filterwarnings('ignore')

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
df=pd.read_csv("/content/drive/MyDrive/aml project/trial/AgrcultureDataset.csv")
df.dtypes

State_Name        object
District_Name     object
Crop_Year          int64
Season            object
Crop              object
Area             float64
Production        object
dtype: object

In [4]:
df.head()

,State_Name,District_Name,Crop_Year,Season,Crop,Area,Production
0,Andaman and Nicobar Islands,NICOBARS,2000,Kharif,Arecanut,1254.0,2000
1,Andaman and Nicobar Islands,NICOBARS,2000,Kharif,Other Kharif pulses,2.0,1
2,Andaman and Nicobar Islands,NICOBARS,2000,Kharif,Rice,102.0,321
3,Andaman and Nicobar Islands,NICOBARS,2000,Whole Year,Banana,176.0,641
4,Andaman and Nicobar Islands,NICOBARS,2000,Whole Year,Cashewnut,720.0,165


In [5]:
#converting production to numeric type
df['Production']=pd.to_numeric(df['Production'],errors='coerce')

In [6]:
#calulation cpi(  )
df['CPI']=df['Production']/df['Area']
df.head()

,State_Name,District_Name,Crop_Year,Season,Crop,Area,Production,CPI
0,Andaman and Nicobar Islands,NICOBARS,2000,Kharif,Arecanut,1254.0,2000.0,1.594896
1,Andaman and Nicobar Islands,NICOBARS,2000,Kharif,Other Kharif pulses,2.0,1.0,0.500000
2,Andaman and Nicobar Islands,NICOBARS,2000,Kharif,Rice,102.0,321.0,3.147059
3,Andaman and Nicobar Islands,NICOBARS,2000,Whole Year,Banana,176.0,641.0,3.642045
4,Andaman and Nicobar Islands,NICOBARS,2000,Whole Year,Cashewnut,720.0,165.0,0.229167


In [7]:
df.shape

(246091, 8)

In [8]:
pd.unique(df['Season'])

array(['Kharif     ', 'Whole Year ', 'Autumn     ', 'Rabi       ',
       'Summer     ', 'Winter     '], dtype=object)

In [9]:
a = pd.unique(df['Crop'])
len(a)

124

In [10]:
#Encoding
le = LabelEncoder()
df['State_Name'] = le.fit_transform(df['State_Name'])
df['District_Name'] = le.fit_transform(df['District_Name'])
df['Season'] = le.fit_transform(df['Season'])
df['Crop'] = le.fit_transform(df['Crop'])

In [11]:
df.isnull().sum()

State_Name          0
District_Name       0
Crop_Year           0
Season              0
Crop                0
Area                0
Production       3727
CPI              3727
dtype: int64

In [12]:
import numpy as np
from sklearn.impute import KNNImputer
kn=KNNImputer(missing_values=np.nan,n_neighbors=7)
df[['Production']]=kn.fit_transform(df[['Production']]).round(0)


In [13]:
df[['CPI']]=kn.fit_transform(df[['CPI']]).round(0)


In [14]:
from sklearn.model_selection import RandomizedSearchCV
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}
print(random_grid)


{'n_estimators': [200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000], 'max_features': ['auto', 'sqrt'], 'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, None], 'min_samples_split': [2, 5, 10], 'min_samples_leaf': [1, 2, 4], 'bootstrap': [True, False]}


In [15]:
a = df.head(200)

In [16]:
#Drop
X = a.drop(['Production','CPI'],axis=1)
Y = a['Production']

In [17]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(X,Y,test_size=0.3)

In [19]:
#RANDOM FOREST REGRESSOR
from sklearn.ensemble import RandomForestRegressor
regressor = RandomForestRegressor()
regressor_random = RandomizedSearchCV(estimator = regressor, param_distributions = random_grid, n_iter = 100, cv = 3, verbose=1, random_state=42, n_jobs = -1)
regressor_random.fit(x_train,y_train)

Fitting 3 folds for each of 100 candidates, totalling 300 fits


RandomizedSearchCV(cv=3, estimator=RandomForestRegressor(), n_iter=100,
                   n_jobs=-1,
                   param_distributions={'bootstrap': [True, False],
                                        'max_depth': [10, 20, 30, 40, 50, 60,
                                                      70, 80, 90, 100, 110,
                                                      None],
                                        'max_features': ['auto', 'sqrt'],
                                        'min_samples_leaf': [1, 2, 4],
                                        'min_samples_split': [2, 5, 10],
                                        'n_estimators': [200, 400, 600, 800,
                                                         1000, 1200, 1400, 1600,
                                                         1800, 2000]},
                   random_state=42, verbose=1)

In [20]:
regressor_random.best_params_


{'n_estimators': 400,
 'min_samples_split': 5,
 'min_samples_leaf': 1,
 'max_features': 'auto',
 'max_depth': 90,
 'bootstrap': False}

In [21]:
regressor = RandomForestRegressor(n_estimators = 200,
 min_samples_split = 5,
 min_samples_leaf = 2,
 max_features = 'auto',
 max_depth = 90,
 bootstrap = False)
regressor.fit(x_train,y_train)

RandomForestRegressor(bootstrap=False, max_depth=90, max_features='auto',
                      min_samples_leaf=2, min_samples_split=5,
                      n_estimators=200)

In [22]:
#Score
modelscore=regressor.score(x_train,y_train)
print(modelscore)

0.9955482229330894


In [23]:
#Score
modelscore=regressor.score(x_train,y_train)
print(modelscore)

0.9955482229330894


In [24]:
#Prediction
y_predict = regressor.predict(x_test)
y_predict

array([4.95250000e+02, 1.50400000e+03, 6.55000000e-01, 2.02666667e+02,
       4.66666667e+01, 6.70800000e+07, 2.42800000e+03, 2.03333333e+01,
       5.38566667e+03, 1.50400000e+03, 2.83150000e+07, 1.08750000e+02,
       4.95250000e+02, 3.17250000e+02, 2.00100000e+03, 2.90000000e+01,
       2.99500000e+02, 1.03333333e+01, 1.51576000e+03, 1.23500000e+02,
       4.80466667e+03, 4.95250000e+02, 7.01333333e+02, 1.88500000e+02,
       2.42800000e+03, 4.66666667e+01, 2.83150000e+07, 1.50400000e+03,
       2.00100000e+03, 2.90000000e+01, 7.45500000e+02, 2.90000000e+01,
       3.08613333e+04, 1.08750000e+02, 1.86933333e+03, 2.03333333e+01,
       9.59500000e+02, 4.66666667e+01, 4.62000000e+02, 2.90000000e+01,
       1.17833333e+03, 1.08750000e+02, 2.42800000e+03, 2.72500000e+02,
       4.80466667e+03, 9.90000000e+01, 2.35755000e+04, 4.25000000e+01,
       6.70800000e+07, 2.42800000e+03, 1.51576000e+03, 2.35755000e+04,
       2.42800000e+03, 4.66666667e+01, 3.17250000e+02, 2.50000000e-01,
      

In [25]:
pred_val = regressor.predict([[0,427,2000,1,95,540]])
pred_val

array([462.])

In [26]:
#New data value
pred_value = regressor.predict([[le.fit_transform(['Andaman and Nicobar Islands']),
                                 le.fit_transform(['NICOBARS']),
                                              2000,
                                 le.fit_transform(['Kharif']),
                                 le.fit_transform(['Rice']),
                                 2000]])
pred_value

array([4804.66666667])

In [ ]:
import pickle

In [ ]:
pickle.dump(regressor,open('/content/drive/MyDrive/aml project/yieldfinal.pkl','wb'))

In [ ]:
import os
import uuid
import urllib
import pickle

import numpy as np
from PIL import Image
from tensorflow.keras.models import load_model

def ValuePredictor(to_predict_list):
	to_predict = np.array(to_predict_list).reshape(1, 6)
	loaded_model = pickle.load(open("/content/drive/MyDrive/aml project/yieldfinal.pkl", "rb"))
	result = loaded_model.predict(to_predict)
	return result[0]


to_predict_list = [2,4,5,6,7,3]
result = ValuePredictor(to_predict_list)	
print("result",result)

result 262159.91


In [ ]:
import joblib
joblib.dump(regressor, '/content/drive/MyDrive/aml project/yield2.joblib')

['/content/drive/MyDrive/aml project/yield2.joblib']

In [ ]:
#calulation cpi(  )


data['CPI']=data['Production']/data['Area']
data.head()

In [ ]:
# Importing libraries

from __future__ import print_function
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import classification_report
from sklearn import metrics
from sklearn import tree
from sklearn.preprocessing import LabelEncoder
import warnings
warnings.filterwarnings('ignore')

In [ ]:
data = pd.read_csv("/content/drive/MyDrive/aml project/crop.csv")

In [ ]:
data["Crop_v"][1]

74

In [ ]:
state = int(13)
district = int(13)
seasons = int(13)
year = int(13)
crops = int(13)
area = int(13)


In [ ]:
import pickle

In [ ]:
loaded_model = pickle.load(open("/content/drive/MyDrive/aml project/yieldfinal.pkl", "rb"))

In [ ]:
y_pred = [[state,district,seasons,year,crops,area]]

result = ValuePredictor(y_pred)
data = pd.read_csv()

In [ ]:
result=[]
for i in range(1,len(data)):
  crop = data["Crop_v"][i]
  y_pred = [[state,district,seasons,year,crop,area]]
  result.append(loaded_model.predict(y_pred))
  #re = zip(i,result)

for i in range(len(result)):
  if(result[i]== max(result)):
    a = data['Crop'][i]


In [ ]:
for i in range(len(result)):
  if(result[i]== max(result)):
    a = data['Crop'][i]
print(a)

Snak Guard
